In [1]:
from flow.multiagent_envs import MultiWaveAttenuationMergePOEnv
from flow.scenarios import MergeScenario
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from ray.tune import run_experiments

import gym, ray, json
from ray.rllib.agents.es import ESAgent, DEFAULT_CONFIG
from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

benchmark_name = 'multi_merge'

In [2]:
alg_run = "ES"
num_rollouts = 1
num_cpus = 3

In [3]:
ray.init(num_cpus=num_cpus, include_webui=False, ignore_reinit_error=True)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-19_05-15-46_702/logs.
Waiting for redis server at 127.0.0.1:47451 to respond...
Waiting for redis server at 127.0.0.1:29135 to respond...
Starting the Plasma object store with 13.355121049 GB memory using /dev/shm.


{'node_ip_address': '169.237.32.118',
 'object_store_addresses': ['/tmp/ray/session_2019-04-19_05-15-46_702/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2019-04-19_05-15-46_702/sockets/raylet'],
 'redis_address': '169.237.32.118:47451',
 'webui_url': ''}

In [4]:
benchmark = __import__(
    "flow.benchmarks.%s" % benchmark_name, fromlist=["flow_params"])
flow_params = benchmark.flow_params
horizon = flow_params['env'].horizon

def make_env(create_env):
    def _thunk():
        env = create_env()
        return env
    return _thunk

create_env, env_name = make_create_env(params=flow_params, version=0)

In [5]:
config = DEFAULT_CONFIG.copy()

In [6]:
config["num_workers"] = min(num_cpus, num_rollouts)
config["episodes_per_batch"] = num_rollouts
config["eval_prob"] = 0.05
# optimal parameters
config["noise_stdev"] = 0.02
config["stepsize"] = 0.02

config["model"]["fcnet_hiddens"] = [100, 50, 25]
config['clip_actions'] = False  # FIXME(ev) temporary ray bug
config["observation_filter"] = "NoFilter"
# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)
config['env_config']['flow_params'] = flow_json
config['env_config']['run'] = alg_run

In [7]:
# Register as rllib env
register_env(env_name, create_env)

In [8]:
agent = ESAgent(config=config, env=env_name)

Created LogSyncer for /headless/ray_results/ES_MultiWaveAttenuationMergePOEnv-v0_2019-04-19_05-15-50zuv5qtm_ -> None
 Starting SUMO on port 43852
2019-04-19 05:15:51,409	INFO es.py:188 -- Creating shared noise table.
2019-04-19 05:16:08,600	INFO es.py:193 -- Creating actors.
2019-04-19 05:16:08,649	INFO es.py:307 -- Collected 0 episodes 0 timesteps so far this iter


RayTaskError: [36mray_Worker:do_rollouts()[39m (pid=742, host=kronos)
  File "/opt/conda/envs/flow/lib/python3.5/site-packages/ray/rllib/agents/es/es.py", line 144, in do_rollouts
    rewards_pos, lengths_pos = self.rollout(timestep_limit)
  File "/opt/conda/envs/flow/lib/python3.5/site-packages/ray/rllib/agents/es/es.py", line 113, in rollout
    add_noise=add_noise)
  File "/opt/conda/envs/flow/lib/python3.5/site-packages/ray/rllib/agents/es/policies.py", line 31, in rollout
    ac = policy.compute(observation, add_noise=add_noise)[0]
  File "/opt/conda/envs/flow/lib/python3.5/site-packages/ray/rllib/agents/es/policies.py", line 72, in compute
    observation = self.observation_filter(observation[None], update=update)
KeyError


In [9]:
env = create_env()
state = env.reset()

 Starting SUMO on port 35229
 Starting SUMO on port 35229


In [10]:
agent.compute_action(state)

KeyError: None

In [ ]:
agent.train()

In [8]:
exp_tag = {
    "run": alg_run,
    "env": env_name,
    "config": {
        **config
    },
    "checkpoint_freq": 1,
    "max_failures": 999,
    "stop": {
        "training_iteration": 3
    },
    "num_samples": 1,
}

In [ ]:
trials = run_experiments({
        flow_params["exp_tag"]: exp_tag
    })

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/3 CPUs, 0/1 GPUs
Memory usage on this node: 4.2/33.4 GB

Created LogSyncer for /headless/ray_results/multi_merge/ES_MultiWaveAttenuationMergePOEnv-v0_0_2019-04-19_04-07-006xgfgkj1 -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 2/3 CPUs, 0/1 GPUs
Memory usage on this node: 4.2/33.4 GB
Result logdir: /headless/ray_results/multi_merge
RUNNING trials:
 - ES_MultiWaveAttenuationMergePOEnv-v0_0:	RUNNING

Error processing event.
Traceback (most recent call last):
  File "/opt/conda/envs/flow/lib/python3.5/site-packages/ray/tune/trial_runner.py", line 261, in _process_events
    result = self.trial_executor.fetch_result(trial)
  File "/opt/conda/envs/flow/lib/python3.5/site-packages/ray/tune/ray_trial_executor.py", line 211, in fetch_result
    result = ray.get(trial_future[0])
  File "/opt/conda/envs/flow/lib/python3.5/site-packages/ray/worker.py", line 2386, in get
    raise value
ray.worker.RayTaskE